# Install

In [19]:
# !rm /workspace/pyproject.toml
! ls /workspace

Dockerfile  circuits-benchmark	    project_template
README.md   cluster_setup.egg-info  quick_experiments


In [1]:
import sys
import importlib.util
import importlib.machinery

# Specify the directory with your local package
local_directory = "/workspace/circuits-benchmark"

# The package name
package_name = "tracr"

# Construct the full path to the package's __init__.py file
package_init_file = f"{local_directory}/{package_name}/__init__.py"

# Load the package from the specified file
spec = importlib.util.spec_from_file_location(package_name, package_init_file)
local_package = importlib.util.module_from_spec(spec)
sys.modules[package_name] = local_package
spec.loader.exec_module(local_package)

# Now you can use the tracr package as usual
import tracr

# Example usage of tracr
print(tracr.__file__)
from tracr.compiler.compiling import TracrOutput

/workspace/circuits-benchmark/tracr/__init__.py


In [2]:
import circuits_benchmark.benchmark.cases.case_3 as case_3

In [17]:
from circuits_benchmark.utils.project_paths import detect_project_root
from circuits_benchmark.utils.project_paths import PROJECT_ROOT
def update_proj_root():
    global PROJECT_ROOT
    print(PROJECT_ROOT)
    PROJECT_ROOT = "/workspace/circuits-benchmark"
    print(PROJECT_ROOT)
update_proj_root()
print(detect_project_root())

# import os


# PROJECT_ROOT: str | None = None


# def detect_project_root() -> str:
#   """
#   Detects the root of the project by looking for a known file in the project.
#   :return: the path to the root of the project.
#   """
#   global PROJECT_ROOT
#   if PROJECT_ROOT is not None:
#     # If the project root has already been detected, return it.
#     return PROJECT_ROOT

#   current_path = os.path.abspath(os.path.curdir)
#   while not os.path.exists(os.path.join(current_path, "pyproject.toml")):
#     print(current_path)
#     current_path = os.path.abspath(os.path.join(current_path, os.pardir))

#   PROJECT_ROOT = current_path

#   return current_path

# print(detect_project_root())

None
/workspace/circuits-benchmark


KeyboardInterrupt: 

In [ ]:
!git lfs install
!git clone https://huggingface.co/cybershiptrooper/InterpBench

# Imports and setup

In [ ]:
import pickle
import torch
import os
import shutil
from transformer_lens import HookedTransformerConfig, HookedTransformer
from transformer_lens import HookedTransformer
from circuits_benchmark.transformers.hooked_tracr_transformer import HookedTracrTransformer

from tokenizers import Tokenizer, models, normalizers, pre_tokenizers, decoders, trainers
from transformers import PreTrainedTokenizerFast


from sae_lens import LanguageModelSAERunnerConfig, SAETrainingRunner
from sae_lens import SAEConfig, SAE, TrainingSAEConfig, TrainingSAE, ActivationsStore, CacheActivationsRunnerConfig, LanguageModelSAERunnerConfig
from sae_lens.training.sae_trainer import SAETrainer

import sae_lens
print(dir(sae_lens.training.sae_trainer))

if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

print("Using device:", device)
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [ ]:
import json
from safetensors.torch import save_file
import wandb
from sae_lens.sae import SAE_CFG_PATH, SAE_WEIGHTS_PATH, SPARSITY_PATH

def save_checkpoint(
        trainer: SAETrainer,
        checkpoint_name: int | str,
        wandb_aliases: list[str] | None = None,
    ) -> str:
        """ Lightly modified from https://github.com/jbloomAus/SAELens/blob/v3.5.0/sae_lens/sae_training_runner.py#L161C5-L210C31 """

        sae = trainer.sae
        os.makedirs(trainer.cfg.checkpoint_path, exist_ok=True)
        checkpoint_path = f"{trainer.cfg.checkpoint_path}/{checkpoint_name}"

        os.makedirs(checkpoint_path, exist_ok=True)

        path = f"{checkpoint_path}"
        os.makedirs(path, exist_ok=True)

        if sae.cfg.normalize_sae_decoder:
            sae.set_decoder_norm_to_unit_norm()
        sae.save_model(path)

        # let's over write the cfg file with the trainer cfg, which is a super set of the original cfg.
        # and should not cause issues but give us more info about SAEs we trained in SAE Lens.
        config = trainer.cfg.to_dict()
        with open(f"{path}/cfg.json", "w") as f:
            json.dump(config, f)
        if trainer.cfg.log_to_wandb:
            print(f'saving {path}')
            wandb.save(path)


        log_feature_sparsities = {"sparsity": trainer.log_feature_sparsity}

        log_feature_sparsity_path = f"{path}/{SPARSITY_PATH}"
        save_file(log_feature_sparsities, log_feature_sparsity_path)

        if trainer.cfg.log_to_wandb and os.path.exists(log_feature_sparsity_path):
            model_artifact = wandb.Artifact(
                f"{sae.get_name()}",
                type="model",
                metadata=dict(trainer.cfg.__dict__),
            )

            model_artifact.add_file(f"{path}/{SAE_WEIGHTS_PATH}")
            model_artifact.add_file(f"{path}/{SAE_CFG_PATH}")

            wandb.log_artifact(model_artifact, aliases=wandb_aliases)

            sparsity_artifact = wandb.Artifact(
                f"{sae.get_name()}_log_feature_sparsity",
                type="log_feature_sparsity",
                metadata=dict(trainer.cfg.__dict__),
            )
            sparsity_artifact.add_file(log_feature_sparsity_path)
            wandb.log_artifact(sparsity_artifact)

        return checkpoint_path

In [ ]:
import circuits_benchmark.benchmark.cases.case_3 as case_3

task = case_3.Case3()
task_idx = task.get_index()

In [ ]:
dir_name = f"InterpBench/{task_idx}"
cfg_dict = pickle.load(open(f"{dir_name}/ll_model_cfg.pkl", "rb"))
cfg = HookedTransformerConfig.from_dict(cfg_dict)
model = HookedTransformer(cfg)
weights = torch.load(f"{dir_name}/ll_model.pth")
model.load_state_dict(weights)

# Load and configure benchmark

In [ ]:
%cd /workspace/circuits-benchmark
# load high level model
from circuits_benchmark.utils.iit import make_iit_hl_model
import circuits_benchmark.utils.iit.correspondence as correspondence
from circuits_benchmark.utils.iit.dataset import get_unique_data
import iit.model_pairs as mp
from datasets import Dataset

def make_model_pair(benchmark_case):
    hl_model = benchmark_case.build_transformer_lens_model()
    hl_model = make_iit_hl_model(hl_model, eval_mode=True)
    tracr_output = benchmark_case.get_tracr_output()
    hl_ll_corr = correspondence.TracrCorrespondence.from_output(
            case=benchmark_case, tracr_output=tracr_output
        )
    model_pair = mp.StrictIITModelPair(hl_model, model, hl_ll_corr)
    return model_pair, hl_model

model_pair, hl_model = make_model_pair(task)

In [ ]:
# Create dataset of case inputs
dataset = get_unique_data(task, max_len=10_000)
tokenized_data = hl_model.map_tracr_input_to_tl_input(dataset.data)

# Convert PyTorch tensors to lists
string_tokens_list = dataset.data.tolist()
tokens_list = tokenized_data.tolist()
labels_list = [str(label) for label in dataset.labels]

# Create a dictionary from the lists
data_dict = {
    "string_tokens": string_tokens_list,
    "tokens": tokens_list,
    "labels": labels_list
}

# Create a Hugging Face dataset
hf_dataset = Dataset.from_dict(data_dict)

print(hf_dataset)
print(hf_dataset[3])

In [ ]:
print(hf_dataset.shape)

In [ ]:
# create tokenizer
# Define your simple vocabulary
vocab = {'BOS': 0, 'UNK': 1, 'a': 2, 'b': 3, 'c': 4, 'x': 5}
# comes from task.get_vocab() and hl_model.map_tracr_input_to_tl_input

# Create a Tokenizer with a WordLevel model
tokenizer = Tokenizer(models.WordLevel(vocab=vocab, unk_token="UNK"))

# Set the normalizer, pre-tokenizer, and decoder
tokenizer.normalizer = normalizers.Sequence([normalizers.Lowercase(), normalizers.StripAccents()])
tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()

# Convert to Hugging Face tokenizer
hf_tokenizer = PreTrainedTokenizerFast(tokenizer_object=tokenizer)

# Add the special tokens to the Hugging Face tokenizer
hf_tokenizer.add_special_tokens({
    'unk_token': 'UNK',
    'bos_token': 'BOS',
    'cls_token': '[CLS]',
    'sep_token': '[SEP]',
    'pad_token': '[PAD]',
    'mask_token': '[MASK]'
})

# Test the tokenizer
encoded = hf_tokenizer.encode("BOS a b c x")
decoded = hf_tokenizer.decode(encoded)
print(f"Encoded: {encoded}")
print(f"Decoded: {decoded}")

In [11]:
model.tokenizer = hf_tokenizer #attach to model.

NameError: name 'hf_tokenizer' is not defined

In [12]:
_, cache = model.run_with_cache(tokenized_data)
output = hl_model(tokenized_data) #TODO: why are these different calls?
print(output[:5], dataset.labels[:5])

NameError: name 'tokenized_data' is not defined

In [13]:
print(cache)

NameError: name 'cache' is not defined

# SAE-lens

In [ ]:
%cd /content
%pwd
%ls

/content
circuits-benchmark/  sample_data/


In [ ]:
class RepeatActivationsStore(ActivationsStore):
    def _get_next_dataset_tokens(self) -> torch.Tensor:
        device = self.device
        if not self.is_dataset_tokenized:
            try:
                s = next(self.iterable_dataset)[self.tokens_column]
            except StopIteration:
                #shuffle self.dataset and restart
                self.iterable_dataset = iter(self.dataset)
                s = next(self.iterable_dataset)[self.tokens_column]
            tokens = (
                self.model.to_tokens(
                    s,
                    truncate=False,
                    move_to_device=True,
                    prepend_bos=self.prepend_bos,
                )
                .squeeze(0)
                .to(device)
            )
            assert (
                len(tokens.shape) == 1
            ), f"tokens.shape should be 1D but was {tokens.shape}"
        else:
            try:
                s = next(self.iterable_dataset)[self.tokens_column]
            except StopIteration:
                #shuffle self.dataset and restart
                self.iterable_dataset = iter(self.dataset)
                s = next(self.iterable_dataset)[self.tokens_column]
            tokens = torch.tensor(
                s,
                dtype=torch.long,
                device=device,
                requires_grad=False,
            )
            if (
                not self.prepend_bos
                and tokens[0] == self.model.tokenizer.bos_token_id  # type: ignore
            ):
                tokens = tokens[1:]
        self.n_dataset_processed += 1
        return tokens

## Residual stream

In [ ]:
L1_coeff = 1e-1
training_tokens = 5_000_000
runner_cfg = LanguageModelSAERunnerConfig(
    # Data Generating Function (Model + Training Distribution)
    model_name = "case3",
    model_class_name = "HookedTransformer",
    hook_name = "blocks.1.hook_resid_pre",
    hook_eval = "NOT_IN_USE",
    hook_layer = 1,
    hook_head_index = None,
    dataset_path = "",
    dataset_trust_remote_code = False,
    streaming = False,
    is_dataset_tokenized = True,
    context_size = 5,
    use_cached_activations = False,
    cached_activations_path = None,  # Defaults to "activations/{dataset}/{model}/{full_hook_name}_{hook_head_index}"

    # SAE Parameters
    d_in = model.cfg.d_model,
    d_sae = None,
    b_dec_init_method = "geometric_median",
    expansion_factor = 4,
    activation_fn = "relu",  # relu, tanh-relu
    normalize_sae_decoder = True,
    noise_scale = 0.0,
    from_pretrained_path = None,
    apply_b_dec_to_input = False,
    decoder_orthogonal_init = False,
    decoder_heuristic_init = False,
    init_encoder_as_decoder_transpose = False,

    # Activation Store Parameters
    n_batches_in_buffer = hf_dataset.shape[0],
    training_tokens = training_tokens,
    finetuning_tokens = 0,
    store_batch_size_prompts = 5,
    normalize_activations = "none",  # none, expected_average_only_in (Anthropic April Update), constant_norm_rescale (Anthropic Feb Update)

    # Misc
    device = device,
    act_store_device = "with_model",  # will be set by post init if with_model
    seed = 42,
    dtype = "float32",  # type: ignore #
    prepend_bos = False,

    # Performance - see compilation section of lm_runner.py for info
    autocast = False,  # autocast to autocast_dtype during training
    autocast_lm = False,  # autocast lm during activation fetching
    compile_llm = False,  # use torch.compile on the LLM
    llm_compilation_mode = None,  # which torch.compile mode to use
    compile_sae = False,  # use torch.compile on the SAE
    sae_compilation_mode = None,

    # Training Parameters

    ## Batch size
    train_batch_size_tokens = 5*100,

    ## Adam
    adam_beta1 = 0.9,
    adam_beta2 = 0.999,

    ## Loss Function
    mse_loss_normalization = None,
    l1_coefficient = L1_coeff,
    lp_norm = 1,
    scale_sparsity_penalty_by_decoder_norm = False,
    l1_warm_up_steps = 0,

    ## Learning Rate Schedule
    lr = 3e-4,
    lr_scheduler_name = "constant",  # constant, cosineannealing, cosineannealingwarmrestarts
    lr_warm_up_steps = 0,
    lr_end = None,  # only used for cosine annealing, default is lr / 10
    lr_decay_steps = 0,
    n_restart_cycles = 1,  # used only for cosineannealingwarmrestarts

    ## FineTuning
    finetuning_method = None,  # scale, decoder or unrotated_decoder

    # Resampling protocol args
    use_ghost_grads = True,  # want to change this to true on some timeline.
    feature_sampling_window = 2000,
    dead_feature_window = 1000,  # unless this window is larger feature sampling,
    dead_feature_threshold = 1e-8,

    # Evals
    n_eval_batches = 10,
    eval_batch_size_prompts = None,  # useful if evals cause OOM

    # WANDB
    log_to_wandb = True,
    log_activations_store_to_wandb = False,
    log_optimizer_state_to_wandb = False,
    wandb_project = "benchmark_saes",
    wandb_id = None,
    run_name = None,
    wandb_entity = None,
    wandb_log_frequency = 10,
    eval_every_n_wandb_logs = 100000000000, # Make this a really big number; currently fails because it tries to compute CE loss.
    # Misc
    resume = False,
    n_checkpoints = 5,
    checkpoint_path = "checkpoints",
    verbose = True,
    model_kwargs = dict(),
    model_from_pretrained_kwargs = dict(),
    sae_lens_version = str(sae_lens.__version__),
    sae_lens_training_version = str(sae_lens.__version__),
)


Run name: 48-L1-0.1-LR-0.0003-Tokens-5.000e+06
n_tokens_per_buffer (millions): 0.032
Lower bound: n_contexts_per_buffer (millions): 0.0064
Total training steps: 10000
Total wandb updates: 1000
n_tokens_per_feature_sampling_window (millions): 5.0
n_tokens_per_dead_feature_window (millions): 2.5
We will reset the sparsity calculation 5 times.
Number tokens in sparsity calculation window: 1.00e+06
Using Ghost Grads.


In [ ]:
from typing import Any, cast

model.tokenizer = hf_tokenizer
store = RepeatActivationsStore.from_config(model, runner_cfg, dataset=hf_dataset)
sae = TrainingSAE(runner_cfg)
trainer = SAETrainer(model, sae, store, save_checkpoint, cfg = runner_cfg)

if runner_cfg.log_to_wandb:
    wandb.init(
        project=runner_cfg.wandb_project,
        config=cast(Any, runner_cfg),
        name=runner_cfg.run_name,
        id=runner_cfg.wandb_id,
    )
trainer.fit()

wandb.finish()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


2000| MSE Loss 0.027 | L1 0.199:  20%|██        | 1000000/5000000 [04:09<15:58, 4171.24it/s]

...

saving checkpoints/ubcrpfrv/4000500


10000| MSE Loss 0.010 | L1 0.125: 100%|██████████| 5000000/5000000 [20:14<00:00, 4128.09it/s]

saving checkpoints/ubcrpfrv/final_5000000


10000| MSE Loss 0.010 | L1 0.125: 100%|██████████| 5000000/5000000 [20:15<00:00, 4114.00it/s]


details/current_l1_coefficient,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
details/current_learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
details/n_training_tokens,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
losses/ghost_grad_loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
losses/l1_loss,█▆▄▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
losses/mse_loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
losses/overall_loss,█▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
metrics/explained_variance,▁▇██████████████████████████████████████
metrics/explained_variance_std,█▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
metrics/l0,█▆▅▅▄▄▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
metrics/mean_log10_feature_sparsity,█▃▂▁▁


## Residual stream

In [ ]:
L1_coeff = 1e-1
training_tokens = 5_000_000
runner_cfg = LanguageModelSAERunnerConfig(
    # Data Generating Function (Model + Training Distribution)
    model_name = "case3",
    model_class_name = "HookedTransformer",
    hook_name = "blocks.0.hook_mlp_out",
    hook_eval = "NOT_IN_USE",
    hook_layer = 0,
    hook_head_index = None,
    dataset_path = "",
    dataset_trust_remote_code = False,
    streaming = False,
    is_dataset_tokenized = True,
    context_size = 5,
    use_cached_activations = False,
    cached_activations_path = None,  # Defaults to "activations/{dataset}/{model}/{full_hook_name}_{hook_head_index}"

    # SAE Parameters
    d_in = model.cfg.d_model,
    d_sae = None,
    b_dec_init_method = "geometric_median",
    expansion_factor = 4,
    activation_fn = "relu",  # relu, tanh-relu
    normalize_sae_decoder = True,
    noise_scale = 0.0,
    from_pretrained_path = None,
    apply_b_dec_to_input = False,
    decoder_orthogonal_init = False,
    decoder_heuristic_init = False,
    init_encoder_as_decoder_transpose = False,

    # Activation Store Parameters
    n_batches_in_buffer = hf_dataset.shape[0],
    training_tokens = training_tokens,
    finetuning_tokens = 0,
    store_batch_size_prompts = 5,
    normalize_activations = "none",  # none, expected_average_only_in (Anthropic April Update), constant_norm_rescale (Anthropic Feb Update)

    # Misc
    device = device,
    act_store_device = "with_model",  # will be set by post init if with_model
    seed = 42,
    dtype = "float32",  # type: ignore #
    prepend_bos = False,

    # Performance - see compilation section of lm_runner.py for info
    autocast = False,  # autocast to autocast_dtype during training
    autocast_lm = False,  # autocast lm during activation fetching
    compile_llm = False,  # use torch.compile on the LLM
    llm_compilation_mode = None,  # which torch.compile mode to use
    compile_sae = False,  # use torch.compile on the SAE
    sae_compilation_mode = None,

    # Training Parameters

    ## Batch size
    train_batch_size_tokens = 5*100,

    ## Adam
    adam_beta1 = 0.9,
    adam_beta2 = 0.999,

    ## Loss Function
    mse_loss_normalization = None,
    l1_coefficient = L1_coeff,
    lp_norm = 1,
    scale_sparsity_penalty_by_decoder_norm = False,
    l1_warm_up_steps = 0,

    ## Learning Rate Schedule
    lr = 3e-4,
    lr_scheduler_name = "constant",  # constant, cosineannealing, cosineannealingwarmrestarts
    lr_warm_up_steps = 0,
    lr_end = None,  # only used for cosine annealing, default is lr / 10
    lr_decay_steps = 0,
    n_restart_cycles = 1,  # used only for cosineannealingwarmrestarts

    ## FineTuning
    finetuning_method = None,  # scale, decoder or unrotated_decoder

    # Resampling protocol args
    use_ghost_grads = True,  # want to change this to true on some timeline.
    feature_sampling_window = 2000,
    dead_feature_window = 1000,  # unless this window is larger feature sampling,
    dead_feature_threshold = 1e-8,

    # Evals
    n_eval_batches = 10,
    eval_batch_size_prompts = None,  # useful if evals cause OOM

    # WANDB
    log_to_wandb = True,
    log_activations_store_to_wandb = False,
    log_optimizer_state_to_wandb = False,
    wandb_project = "benchmark_saes",
    wandb_id = None,
    run_name = None,
    wandb_entity = None,
    wandb_log_frequency = 10,
    eval_every_n_wandb_logs = 100000000000, # Make this a really big number; currently fails because it tries to compute CE loss.
    # Misc
    resume = False,
    n_checkpoints = 5,
    checkpoint_path = "checkpoints",
    verbose = True,
    model_kwargs = dict(),
    model_from_pretrained_kwargs = dict(),
    sae_lens_version = str(sae_lens.__version__),
    sae_lens_training_version = str(sae_lens.__version__),
)


Run name: 48-L1-0.1-LR-0.0003-Tokens-5.000e+06
n_tokens_per_buffer (millions): 0.032
Lower bound: n_contexts_per_buffer (millions): 0.0064
Total training steps: 10000
Total wandb updates: 1000
n_tokens_per_feature_sampling_window (millions): 5.0
n_tokens_per_dead_feature_window (millions): 2.5
We will reset the sparsity calculation 5 times.
Number tokens in sparsity calculation window: 1.00e+06
Using Ghost Grads.


In [ ]:
import shutil

In [ ]:
from typing import Any, cast

model.tokenizer = hf_tokenizer
store = RepeatActivationsStore.from_config(model, runner_cfg, dataset=hf_dataset)
sae = TrainingSAE(runner_cfg)
trainer = SAETrainer(model, sae, store, save_checkpoint, cfg = runner_cfg)

if runner_cfg.log_to_wandb:
    wandb.init(
        project=runner_cfg.wandb_project,
        config=cast(Any, runner_cfg),
        name=runner_cfg.run_name,
        id=runner_cfg.wandb_id,
    )
trainer.fit()

wandb.finish()

wandb: Currently logged in as: evanhanders. Use `wandb login --relogin` to force relogin


2000| MSE Loss 0.028 | L1 0.134:  20%|██        | 1000000/5000000 [03:06<12:01, 5541.47it/s]

saving checkpoints/3p1891vk/1000500


4000| MSE Loss 0.018 | L1 0.098:  40%|████      | 2000000/5000000 [06:07<08:50, 5658.93it/s]

saving checkpoints/3p1891vk/2000500


...

saving checkpoints/3p1891vk/4000500


10000| MSE Loss 0.011 | L1 0.091: 100%|██████████| 5000000/5000000 [15:16<00:00, 5508.34it/s]

saving checkpoints/3p1891vk/final_5000000


10000| MSE Loss 0.011 | L1 0.091: 100%|██████████| 5000000/5000000 [15:16<00:00, 5453.94it/s]


details/current_l1_coefficient,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
details/current_learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
details/n_training_tokens,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
losses/ghost_grad_loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
losses/l1_loss,█▅▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
losses/mse_loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
losses/overall_loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
metrics/explained_variance,▁███████████████████████████████████████
metrics/explained_variance_std,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
metrics/l0,█▆▄▄▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
metrics/mean_log10_feature_sparsity,█▃▂▁▁


In [ ]:
import wandb

# Initialize the wandb API
api = wandb.Api()



# Get the artifact from the old run
artifact = api.artifact('evanhanders/benchmark_saes/sae_case3_blocks.0.hook_mlp_out_48:v32')

# Download the artifact to a specified directory
artifact_dir = artifact.download("/content/sample_data/mlp_out_0")


artifact = api.artifact('evanhanders/benchmark_saes/sae_case3_blocks.1.hook_resid_pre_48:v5')

# Download the artifact to a specified directory
artifact_dir = artifact.download("/content/sample_data/hook_resid_pre_1")

wandb:   2 of 2 files downloaded.  
wandb:   2 of 2 files downloaded.  
